In [1]:
import pandas as pd
import psycopg2
import user_credentials

In [2]:
# Connect to the climbing_db database
database_name = "climbing_db"
try:
    conn = psycopg2.connect(database = database_name, user = user_credentials.username, password = user_credentials.password, 
                            host = "localhost", port = "5432")
    cursor = conn.cursor()
except psycopg2.errors.OperationalError:
    print("Database connection not successful")

In [3]:
# Query routes data
cursor.execute('''
SELECT c.country, r.crag, r.sector, r.name, r.grade_mean, r.rating_tot, r.style
FROM countries AS c
JOIN routes AS r ON c.country_id = r.country
''')
routes_data = cursor.fetchall()

# Extract the data into a dataframe
routes_df = pd.DataFrame(routes_data, columns=['country', 'crag', 'sector', 'name', 'grade', 'rating', 'style'])

In [4]:
# Query grades data
cursor.execute('SELECT * FROM grades')
grades_data = cursor.fetchall()

# Extract the data into a dataframe
grades_df = pd.DataFrame(grades_data, columns=['id', 'french', 'yds', 'v'])
grades_df.set_index('id', inplace=True)

In [5]:
cursor.close()
conn.close()

In [6]:
county = ''
grade_lower = 'V5'
grade_upper = 100
style = 'Neutral'

In [7]:
if not isinstance(grade_lower, int):
    grade_scales = ['french', 'yds', 'v']
    selected_scale = ''
    if '.' in grade_lower:
        selected_scale = 'yds'
    elif 'v' in grade_lower.lower():
         selected_scale = 'v'
    else:
         selected_scale = 'french'
    grade_scales.remove(selected_scale)
    grades_df.drop(grade_scales, axis=1, inplace=True)
    df = pd.merge(routes_df, grades_df, left_on='grade', right_on='id')
    df.drop('grade', axis=1, inplace=True)
    df.rename(columns={selected_scale : 'grade'}, inplace=True)

In [8]:
df

,country,crag,sector,name,rating,style,grade
0,Andorra,Montserrat,Prohibitivo,Diagonal De La X,-0.045211,Short,V3
1,Andorra,Montserrat,Prohibitivo,Mehir,0.116464,Short,V3
2,Andorra,Montserrat,Prohibitivo,Pas De La Discordia,0.178722,Neutral,V3
3,Argentina,Bariloche,Pared Blanca,Barbaroja,-0.164902,Neutral,V3
4,Argentina,Cerro Ventana,Abajo,Chorro Negro,-0.116780,Neutral,V3
...,...,...,...,...,...,...,...
55853,France,Ceuse,Biographie,Realization,-0.166750,Neutral,V15
55854,Norway,Flatanger,Hanshallaren,Thors Hammer,0.091883,Short,V15
55855,Spain,Santa Linya,Cova Gran,Seleccio Anal,-0.076181,Short,V14
55856,Italy,Arco,Massone,Ultimatum,-0.104177,Neutral,V14
